## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True

tradelogger.setLevel(logging.INFO)

In [2]:
tradelogger.info("Let's do trading")

##  <span style="color:green">Trading Strategy </span>

In [6]:
# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
BBT2 = 0
BBM2 = 0
BBB2 = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0
buy_df = pd.DataFrame()
sell_df = pd.DataFrame()
fastk = 0
fastd = 0

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1

#TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)) & (ROC(CLOSE) >1) )
#TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2))  & (ROC(CLOSE) <1))

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
toTick = lambda x,n=5: np.round((np.floor(x *100)+n-1)/n)*n/100
profit = 0
stoploss = 0
openpositions = 0
target = 0

from talib import CDLLONGLEGGEDDOJI, CDLMARUBOZU, CDLLONGLINE, CDLDARKCLOUDCOVER, CDLPIERCING, CDLBELTHOLD, CDLCLOSINGMARUBOZU

class algoTrade:
    
   
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50, fastk, fastd
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,1.75,1.75,0)
        BBT2, BBM2, BBB2 = BBANDS( CLOSE, 20,3,3,0)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
        fastk, fastd = STOCHF(HIGH, LOW, CLOSE)
        iRSI = RSI(CLOSE, timeperiod=20)
        #logging.info(price)
    
    # Long Strategies
    def long_bb(self):
        temp_df = 0
        temp_df = temp_df | (REF(fastk,1) <=20)&(fastk>20)
        temp_df = temp_df | CROSSOVER(fastk, fastd)
        temp_df = temp_df | (REF(iRSI,1) <=30)&(iRSI>30)
        temp_df = temp_df & ( CROSSOVER(CLOSE, BBB) & (CDLSPINNINGTOP(OPEN, HIGH, LOW, CLOSE) == 0))
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_bb(self):
        temp_df = 0
        temp_df = temp_df | (REF(fastk,1) >=80)&(fastk<80) & CROSSOVER(fastd, fastk)
        #temp_df = temp_df | CROSSOVER(fastd, fastk)
        temp_df = temp_df | (REF(iRSI,1) >=70)&(iRSI<70)
        temp_df = temp_df & CROSSOVER(BBT, CLOSE)
        return pd.DataFrame( temp_df, columns=["sell"])
    
    
    def long_pa(self):
        temp_df = 0
        temp_df =   (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
        temp_df = temp_df | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
        temp_df = temp_df | (CDLPIERCING(OPEN, HIGH, LOW, CLOSE) == 100)
        temp_df = temp_df | (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) == 100)
        temp_df = temp_df | (CDLDOJISTAR(OPEN, HIGH, LOW, CLOSE) != 0)
        temp_df = temp_df | ((CDLMORNINGSTAR(OPEN, HIGH, LOW, CLOSE) == 100) & (CDLMORNINGDOJISTAR(OPEN, HIGH, LOW, CLOSE) == 100))
        temp_df = temp_df | (CDLMARUBOZU(OPEN, HIGH, LOW, CLOSE) == 100) | (CDLCLOSINGMARUBOZU(OPEN, HIGH, LOW, CLOSE) ==100)
        temp_df = temp_df | (CDLBELTHOLD(OPEN, HIGH, LOW, CLOSE) == 100)
        
        temp_df = REF(TREND_DOWN(CLOSE,4),1) & temp_df
        
        return pd.DataFrame(temp_df, columns=["buy"])
    def short_pa(self):
        temp_df = 0
        
        temp_df =   (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
        temp_df = temp_df | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
        temp_df = temp_df | (CDLDARKCLOUDCOVER(OPEN, HIGH, LOW, CLOSE) == -100)
        temp_df = temp_df | (CDLDOJISTAR(OPEN, HIGH, LOW, CLOSE) != 0)
        temp_df = temp_df | (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
        temp_df = temp_df | ((CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) == -100) & (CDLEVENINGDOJISTAR(OPEN, HIGH, LOW, CLOSE) == -100))
        temp_df = temp_df | (CDLMARUBOZU(OPEN, HIGH, LOW, CLOSE) == -100) | (CDLCLOSINGMARUBOZU(OPEN, HIGH, LOW, CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(OPEN, HIGH, LOW, CLOSE) == -100)
        
        temp_df = REF(TREND_UP(CLOSE,4),1) & temp_df
        
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_macd(self):
        temp_df = 1
        #temp_df = temp_df | CROSSOVER(iMACD, iMACDsignal)
        temp_df = REF(TREND_DOWN(CLOSE,15),1) & temp_df
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_macd(self):
        temp_df = 1
        #temp_df = temp_df | CROSSOVER(iMACDsignal, iMACD)
        temp_df = REF(TREND_UP(CLOSE,15),1) & temp_df
        return pd.DataFrame( temp_df , columns=["sell"])
  
    def tradeDecision(self):
        buy = self.long_bb() & self.long_pa()
        sell = self.short_bb()# & self.short_pa()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = -1 * CLOSE
        sell['close'] = CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        
        #temp_df = pd.DataFrame(buy.close)
        
        temp_df = pd.concat([buy.close, sell.close]).sort_index()
        #temp_df.append(buy.close)
        #temp_df = temp_df.append(sell.close)
        logging.debug(temp_df)
        return (buy,sell)
    
    
    # do not set timestamp for live trading
    def tradeBot(self, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        global STATE, profit, target, stoploss, papertrade, openpositions
        
        timeNow = timestamp        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)
        
        #atr = ATR(HIGH, LOW, CLOSE)[-1]
        atr = CLOSE[-1]*0.025

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        longSL = toTick(CLOSE[-1] - atr)
        shortSL = toTick(CLOSE[-1]+atr)
        longTarget = toTick(CLOSE[-1] + atr) 
        shortTarget = toTick(CLOSE[-1]- atr)

        check_trade = lambda x, dt1, dt2: True if (x.shape[0] > 0) and ((x.index[-1]==dt1) or (x.index[-1]==dt2)) else False
        print_trade = lambda x: logging.debug(x) if (x.empty != True) else False
        
        if papertrade == True and STATE != "":
            if STATE == "SELL":
                if LOW[-1]<=stoploss:
                    temp_price = stoploss
                    STATE = ""
                    openpositions = openpositions - 1
                    tradelogger.info("Squareoff Stoploss: sell: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                elif HIGH[-1]>=target:
                    temp_price = target
                    STATE = ""
                    openpositions = openpositions - 1
                    tradelogger.info("Squareoff Target: sell: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
            elif STATE == "BUY":
                if HIGH[-1]>=stoploss:
                    temp_price = -1 * stoploss
                    STATE = ""
                    openpositions = openpositions + 1
                    tradelogger.info("Squareoff Stoploss: buy: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                elif LOW[-1]<=target:
                    temp_price = -1 * target
                    STATE = ""
                    openpositions = openpositions + 1
                    tradelogger.info("Squareoff Target: buy: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                
            
        
        buy, sell = self.tradeDecision()
        
        tradelogger.debug("STATE:" + STATE)
        tradelogger.debug(buy)
        tradelogger.debug(sell)
        
        
        tradelogger.debug(timeNow)
        tradelogger.debug(delT1)
        tradelogger.debug(delT2)
        
        # BUY in BUY state, SELL in STATE state
        if check_trade(buy, delT1, delT2) and (STATE != "SELL"):            
            tradelogger.info("buy:"+str(buy.index[-1]))
            openpositions = openpositions + 1
            stoploss = longSL
            target = longTarget
            tradelogger.debug(openpositions)
            #tradelogger.info(atr)
            ##### Order Placement #####
            buy_bo(symbol, price, trigger, longSL, longTarget, quantity=1)
            #buy_slm(symbol, price, stoploss, quantity=1)
            STATE = "SELL"
            return -1 * price
            
        elif check_trade(sell, delT1, delT2) and (STATE!="BUY"):                
            tradelogger.info("sell:"+str(sell.index[-1]))
            openpositions = openpositions - 1
            stoploss = shortSL
            target = shortTarget
            tradelogger.debug(openpositions)
            #tradelogger.info(atr)
            ##### Order Placement #####
            sell_bo(symbol, price, trigger, shortSL, shortTarget, quantity=1)
            #sell_slm(symbol, price, stoploss, quantity=1)
            STATE = "BUY"
            return price
        
        return 0

#portfolio = pd.DataFrame(downloadlist)

if mode == "Algo" and False:
    x = 'BHARTIARTL'
    
    toDate = dt.datetime(2019,4,22,15,15)
    fromDate =   getFromDate(toDate)
    STATE = "BUY"
    #logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame([x])
    papertrade = True
    
    temp_data = getData(x, fromDate, toDate, "minute")
    
    a = algoTrade(temp_data)
    
    buy, sell = a.tradeDecision()
    
    #print(buy, sell)
    
    fig = createPlot(x)
    plot_set1(temp_data)
    
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)

In [ ]:
if mode == "Algo":
    x = 'CIPLA'
    
    toDate = dt.datetime(2019,4,23,15,15)
    fromDate = getFromDate(toDate)
    STATE = "BUY"
    #logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame([x])
    papertrade = True
    
    temp_data = getData(x, fromDate, toDate, "minute")
    
    a = algoTrade(temp_data)
    
    buy, sell = a.tradeDecision()
    
    fig = createPlot(x)
    plot_set1(temp_data)
    
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)

In [66]:
#gc.disable()

In [34]:
#gc.get_count()

(40, 11, 55)

In [ ]:
tradelogger.setLevel(logging.INFO)
if mode == "Algo":
    portfolio = downloadlist
    portfolio = ['CIPLA']
    toDate = dt.datetime(2019,4,22,15,15)
    fromDate = getFromDate(toDate)
    STATE = ""
    stoploss = 0
    target = 0
    openpositions = 0
    portfolio = pd.DataFrame(portfolio)
    papertrade = True
    backtest(portfolio, fromDate, toDate)

- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick


No new open position after 3:00. Only close open positions

before 9:15: Go long based on one day candle
9:15 - 11:30: MACD or Stochastic+BB: One trigger, and atleast one confirmation
11:30 - 2:00: No trade untill and unless there is a golden cross or death cross
2:00 - 2:45: Go long if price is smaller than VWAP, Go short if price is higher than VWAP
2:00 - 2:45: Go long or short only if price crosses SMA(200)
